# Librerias Necesarias

In [1]:
#!pip install umap
#!pip install mlxtend
#!!pip install 'pami[gpu]'

In [2]:
import gc
import matplotlib.pyplot as plt
import numpy as np
from PAMI.frequentPattern.basic import FPGrowth
import pandas as pd
import pickle
import seaborn as sns
import warnings

In [3]:
import sys
sys.setrecursionlimit(16385)

In [4]:
#Libreamos espacio en la GPU
gc.collect()

0

In [5]:
#Para visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [6]:
#Eliminar las advertencias 
warnings.filterwarnings("ignore")

# Lectura de Datos 
Cargamos los datasets resultantes

In [7]:
base_path = "./ConjuntosDatos/"
endutih_path = base_path + "ENDUTIH2020/conjuntos_de_datos/"

In [8]:
endutih_vivhogar = pd.read_csv(endutih_path+"viv_hog_agrupados.csv", dtype={"Grupo":str})
endutih_res = pd.read_csv(endutih_path+"residente_agrupados.csv", dtype={"Grupo":str})
endutih_usr = pd.read_csv(endutih_path+"usuario_agrupados.csv", low_memory=False, dtype={"Grupo":str})
endutih_usr2 = pd.read_csv(endutih_path+"usuario2_agrupados.csv", low_memory=False, dtype={"Grupo":str})
estados = pd.read_csv(base_path+"CodigoEdos.csv")
columnas_sociotec = pd.read_csv("./columnas_clustering/cols_clus_medio_bajo.csv").columns

endutih_completa = pd.merge(endutih_vivhogar,endutih_res.drop(["UPM_DIS","ESTRATO","ENT","DOMINIO", "EST_DIS", "TLOC","Grupo"], axis=1), on = ["UPM","VIV_SEL", "HOGAR"])
endutih_completa = pd.merge(endutih_completa, estados["Codigo ENT".split()], left_on=["ENT"], right_on="Codigo", suffixes=("_orig","")).drop("Codigo ENT_orig".split(), axis=1)

In [9]:
endutih_completa.head()

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT
0,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,1,1,2,54,27.0,3.0,2.0,0.0,0.0,9.0,9.0,1.0,6.0,4.0,NaN,575,B.C.
1,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,2,3,2,22,7.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,NaN,1.0,575,B.C.
2,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,1,1,2,36,29.0,6.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.
3,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,2,3,1,20,29.0,3.0,2.0,6.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.
4,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,3,3,2,19,20.0,2.0,2.0,6.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.


In [10]:
resumen_idtmex = pd.read_csv(base_path+"long_IDTMex_desglosado_dominio_grupos_edo.csv", index_col=0, dtype={"ESTRATO":str, "Grupo":str})
resumen_idtmex

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex
2,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610
3,2,R,21,Ags.,1.000000,0.493244,0.506113,6.985203
4,2,R,22,Ags.,0.639535,0.606965,0.521212,6.028424
5,2,R,23,Ags.,0.500000,0.666667,0.333333,5.333333
6,2,U,20,Ags.,0.534371,0.415015,0.463455,4.724455
...,...,...,...,...,...,...,...,...
632,3,U,32,Zac.,0.750000,0.500000,0.500000,6.000000
633,3,U,33,Zac.,0.719707,0.623086,0.640362,6.651896
634,3,U,34,Zac.,0.847005,0.586430,0.666667,7.067073
637,4,U,42,Zac.,0.805036,0.756772,0.789014,7.825261


In [11]:
resumen_idtmex.describe(include="all")

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex
count,520,520,520,520,520.000000,520.000000,520.000000,520.000000
unique,4,2,15,32,NaN,NaN,NaN,NaN
top,2,U,20,B.C.S,NaN,NaN,NaN,NaN
freq,192,418,62,20,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.683853,0.565991,0.618888,6.237152
std,NaN,NaN,NaN,NaN,0.212327,0.207891,0.191084,1.849581
min,NaN,NaN,NaN,NaN,0.123636,0.000000,0.166667,1.511805
25%,NaN,NaN,NaN,NaN,0.510543,0.445805,0.463121,4.934255
50%,NaN,NaN,NaN,NaN,0.701144,0.598388,0.596687,6.567132
75%,NaN,NaN,NaN,NaN,0.856716,0.668162,0.777778,7.666667


# Seleccion de grupos a analizar

In [12]:
grupos_bajos = resumen_idtmex[resumen_idtmex.IDTMex<4.934255]
grupos_bajos

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex
2,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610
6,2,U,20,Ags.,0.534371,0.415015,0.463455,4.724455
9,2,U,23,Ags.,0.518510,0.343316,0.474209,4.395722
20,1,R,11,B.C.,0.370303,0.270911,0.423337,3.411529
22,2,R,20,B.C.,0.512853,0.447157,0.431790,4.703621
...,...,...,...,...,...,...,...,...
620,1,R,11,Zac.,0.351333,0.243726,0.352223,3.084682
622,2,R,20,Zac.,0.430278,0.310738,0.409293,3.782650
625,2,R,23,Zac.,0.339466,0.279458,0.596528,3.668753
629,2,U,23,Zac.,0.369048,0.284516,0.264151,3.142558


In [13]:
grupos_bajos.describe()

,acceso,uso,aptitudes,IDTMex
count,130.000000,130.000000,130.000000,130.000000
mean,0.406619,0.292235,0.428364,3.652147
std,0.097911,0.130086,0.099851,0.905645
min,0.123636,0.000000,0.166667,1.511805
25%,0.334449,0.221024,0.364925,2.960097
50%,0.415122,0.299599,0.415440,3.780214
75%,0.500000,0.381221,0.463388,4.503766
max,0.582504,0.592507,0.833333,4.915124


In [14]:
patrones_idtmex_bajo=dict()

In [37]:
i = 0

In [39]:
grupos_bajos.iloc[[i]]

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex
2,2,R,20,Ags.,0.526023,0.418287,0.463184,4.70361


In [40]:
dominio, grupo, entidad = grupos_bajos.iloc[i]["DOMINIO Grupo ENT".split()]
dominio, grupo, entidad

('R', '20', 'Ags.')

In [41]:
base_tmp = endutih_completa[(endutih_completa.DOMINIO==dominio)&(endutih_completa.Grupo==grupo)&(endutih_completa.ENT==entidad)]
base_tmp

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT
199587,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,272,Ags.
199588,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,2,5,2,48,1.0,12.0,2.0,99.0,9.0,1.0,1.0,1.0,1.0,NaN,4.0,272,Ags.
199589,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,3,5,2,82,13.0,12.0,2.0,2.0,6.0,2.0,2.0,2.0,1.0,NaN,4.0,272,Ags.
199590,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,4,3,2,19,21.0,2.0,2.0,6.0,3.0,1.0,1.0,1.0,6.0,4.0,NaN,272,Ags.
199591,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,5,3,1,13,6.0,10.0,1.0,3.0,1.0,1.0,1.0,1.0,5.0,4.0,NaN,272,Ags.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200977,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,1,1,1,38,25.0,7.0,2.0,2.0,5.0,2.0,1.0,1.0,1.0,NaN,3.0,286,Ags.
200978,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,2,2,2,37,22.0,4.0,2.0,2.0,6.0,2.0,2.0,1.0,1.0,NaN,1.0,286,Ags.
200979,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,3,3,2,16,24.0,1.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,3.0,286,Ags.
200980,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,4,3,1,16,1.0,11.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,3.0,286,Ags.


In [43]:
%%time
hogares_extended = []
for i in base_tmp.index:
    reps = base_tmp.loc[i,"FAC_HOGAR"]
    df_temp = base_tmp.loc[[i]].copy()
    df_temp["FAC_HOGAR".split()] = 1
    tot_trans = np.tile(df_temp, (reps,1))
    hogares_extended.append(pd.DataFrame(tot_trans))
    del df_temp
base_tmp_ext = pd.concat(hogares_extended, ignore_index=True)
base_tmp_ext.columns = base_tmp.columns
del hogares_extended #Liberar espacio
base_tmp_ext

CPU times: total: 1.22 s
Wall time: 1.22 s


,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT
0,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,1,Ags.
1,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,1,Ags.
2,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,1,Ags.
3,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,1,Ags.
4,3,1,1,1,1,1,1,7,1,1.0,272,1,2,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,250.0,1.0,280.0,1.0,3.0,1.0,272,R,4,2,20,160044,292,5,1,1,1,1,1,48,99.0,99.0,2.0,2.0,6.0,2.0,1.0,1.0,1.0,NaN,4.0,1,Ags.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349692,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,5,3,2,6,27.0,10.0,1.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,1,Ags.
349693,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,5,3,2,6,27.0,10.0,1.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,1,Ags.
349694,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,5,3,2,6,27.0,10.0,1.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,1,Ags.
349695,2,2,1,1,2,1,2,5,1,1.0,286,1,1,1,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,286,R,4,2,20,160027,291,5,1,20,5,3,2,6,27.0,10.0,1.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,1,Ags.


In [44]:
%%time

base_tmp_ext_ohe = pd.get_dummies(base_tmp_ext[columnas_sociotec], columns=columnas_sociotec, drop_first=False)
gc.collect()


CPU times: total: 3.38 s
Wall time: 3.38 s


204

In [ ]:
transactions = []
for row in base_tmp_ext_ohe.itertuples(index=False):
    transaction = set(col for col, value in zip(base_tmp_ext_ohe.columns, row) if value == 1)
    transactions.append(transaction)


In [ ]:
obj = FPGrowth.FPGrowth(iFile=base_tmp_ext_ohe, minSup=0.25)    #initialize
obj.mine()            #start the mining process

obj.save('test.txt') #save the patterns


frequentPatternsDF= obj.getPatternsAsDataFrame() #get the generated frequent patterns as a dataframe
print('Total No of patterns: ' + str(len(frequentPatternsDF))) #print the total number of patterns
print('Runtime: ' + str(obj.getRuntime())) #measure the runtime

print('Memory (RSS): ' + str(obj.getMemoryRSS()))
print('Memory (USS): ' + str(obj.getMemoryUSS()))

<bound method FPGrowth.getPatterns of <PAMI.frequentPattern.basic.FPGrowth.FPGrowth object at 0x000002100DACD510>>